#### Basic request

    'https://api.vk.com/method/wall.get?owner_id=-GROUP_ID&access_token=TOKEN&v=5.60'

#### Change date format from timestamp to datetime
    
    import datetime
    
    datetime.datetime.fromtimestamp(int( DATE )).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
import vk
import json
import time
import datetime
from tqdm import tqdm, tqdm_notebook, trange
import pandas as pd

In [ ]:
#maximum number of requests = 5000

def get_wall(api, pub_id, count, offset=None):
    return api.wall.get(owner_id = pub_id, offset = offset, count = count)

def data(l):
    d = []
    for e in l:
        if e['text'] != '':
            DATE = e['date']
            D = datetime.datetime.fromtimestamp(int(DATE)).strftime('%Y-%m-%d %H:%M:%S')
            T = xa0(e['text'])
            b = {'date': D, 'text': T}
            d.append(b)
    return d

def xa0(string):
    """
    Hey, mister \xa0, GTFOH! 
    """
    if '\xa0' in string:
        string = string.replace('\xa0', ' ')
    return string

def main():
    TOKKEN = str(input("Paste your tokken: "))
    GROUP_ID = int(input("Paste group id: "))
    NAME = input("Paste file name: ")
    
    session = vk.Session(access_token=TOKKEN)
    api = vk.API(session, v='5.60')

    r = get_wall(api, GROUP_ID, 100)

    main_count = r['count']
    num_of_cycles = main_count//100
    remainder = main_count%100
    offset = 100
    
    list_of_posts = data(r['items'])

    with tqdm_notebook(total=num_of_cycles-1) as pbar:
        for iteration in range(num_of_cycles-1):
            current_r = get_wall(api, GROUP_ID, 100, offset=offset)
            list_of_posts.extend(data(current_r['items']))
            offset+=100
            time.sleep(3)
            pbar.update()

    if remainder > 0:
        print('Allez...')
        remainder_r = get_wall(api, GROUP_ID, remainder, offset=offset)
        list_of_posts.extend(data(remainder_r['items']))
        print('oop!')

    with open(NAME+'.json', 'w') as f:
        json.dump(list_of_posts, f)

    print('Done!')

main()